In [18]:
import numpy as np
import matplotlib.pyplot as plt
import random
import pandas as pd
import math
import os

I395_A = [[694, 1], [1416, 1], [1779, 1], [2342, 1], [2817, 1], [3330, 1], [3758, 1], [4261, 1], [4715, 1], [5385, 1], [5741, 1], [6398, 1], [7299, 1], [7832, 1], [8171, 1], [8692, 1], [8885, 1], [9409, 1], [9505, 1], [10150, 1], [10182, 1]]
I395_S = [(7128, 1), (2364, 1), (1635, 1), (4138, 1), (3890, 1), (1333, 1), (5680, 1), (9553, 1), (9388, 1), (9454, 1), (4519, 1), (2662, 1), (4586, 1), (5289, 1), (1108, 1), (9914, 1), (7571, 1), (2874, 1), (9231, 1), (9229, 1), (5281, 1)]
I395_L = [(532, 1), (638, 1), (7082, 1), (4386, 1), (10022, 1), (4579, 1), (8194, 1), (9008, 1), (9230, 1), (6328, 1), (7804, 1), (6901, 1), (7581, 1), (7581, 1), (6901, 1), (7731, 1), (3450, 1), (8412, 1), (8884, 1), (7334, 1), (5284, 1), (8029, 1), (9208, 1), (7731, 1)]

I9094_A = [[5366, 1], [195, 2], [286, 3]]
I9094_S = [(3799, 1), (174, 2), (232, 3)]
I9094_L = [(5012, 1), (5266, 1), (442, 3)]

I294l1_A = [[8, 1], [9, 1], [12, 1], [33, 3], [40, 3], [41, 3], [3, 7], [11, 7], [17, 7], [51, 8], [62, 8], [65, 8], [24, 9], [28, 9], [30, 9], [19, 11], [22, 11], [35, 11], [18, 19], [25, 19], [48, 20], [50, 20], [54, 20], [13, 21]]
I294l1_S = [(727, 7), (5557, 8), (46, 3), (17, 1), (33, 1), (65, 1), (24, 3), (575, 3), (628, 3),(15, 7), (24, 7), (67, 7), (81, 8), (145, 8), (197, 8), (32, 9), (48, 9), (778, 9), (38, 11), (132, 11), (497, 11), (321, 19), (1222, 19),(40, 20),(65, 20),(600, 20),(18, 21)]
I294l1_L = [(603, 11), (1115, 20), (4681, 8), (18, 1), (116, 1),(202, 1), (7, 3), (30, 3),(65, 3), (26, 7), (34, 7), (128, 7), (45, 8), (93, 8), (539, 8), (462, 9), (2008, 9), (3062, 9), (12, 11), (39, 11), (121, 11), (4, 19), (108, 19), (55, 20), (806, 20), (1731, 20), (34, 21)]

I294l2_A = [[462, 5], [107, 23], [291, 28], [90, 29], [118, 30], [231, 31], [181, 33], [218, 35], [46, 36], [72, 38], [211, 41], [229, 42]]
I294l2_S = [[143, 5], [18, 23], [234, 28], [38, 29], [81, 30], [352, 31], [317, 33], [2, 35], [176, 36], [292, 38], [89, 41], [299, 42]]
I294l2_L = [(260, 5), (111, 23), (156, 28), (102, 29), (30, 30), (35, 31), (25, 33), (8, 35), (71, 36), (86, 38), (364, 41), (9, 42)]

population_size, num_generations, mutation_rate = 40, 80, 0.1 # Simulation parameters
T, a, b, v0, delta, so = None, None, None, None, None, None
most_leading_leader_id = None

In [20]:
def generate_vehicle_groups(datasets):
    """
    Generate vehicle group lists dynamically from datasets.
    Ensures each (ID, run_index) pair is unique.
    Returns a dictionary with keys as group names (e.g., 'I395_A', 'I9094_S') and values as lists of [ID, run-index].
    """
    group_lists = {}
    
    for dataset_key, dataset_path in datasets.items():
        # Read the dataset
        df = pd.read_csv(dataset_path)
        df = df.sort_values(by='time')
        df['time'] = df['time'].round(1)
        
        group_A, group_S, group_L = set(), set(), set()  # Use sets to ensure uniqueness
        
        if dataset_key == "I395":
            # Criteria for I395
            for _, row in df.iterrows():
                if row['type-most-common'] == 2 or row['type-most-common'] == 3:
                    group_L.add((row['ID'], row['run-index']))
                elif row['type-most-common'] == 1:
                    group_S.add((row['ID'], row['run-index']))
                elif row['type-most-common'] == 4:
                    group_A.add((row['ID'], row['run-index']))
        else:
            # Determine which column indicates AV/ACC status
            av_column = None
            if "ACC" in df.columns:
                av_column = "ACC"
            elif "AV" in df.columns:
                av_column = "AV"
            
            # Criteria for other datasets
            for _, row in df.iterrows():
                if av_column and str(row[av_column]).strip().lower() == "yes":
                    group_A.add((row['ID'], row['run-index']))
                elif row['type-most-common'] == "small-vehicle":
                    group_S.add((row['ID'], row['run-index']))
                elif row['type-most-common'] == "large-vehicle":
                    group_L.add((row['ID'], row['run-index']))
        
        # Convert sets back to lists
        group_lists[f"{dataset_key}_A"] = [list(item) for item in group_A]
        group_lists[f"{dataset_key}_S"] = [list(item) for item in group_S]
        group_lists[f"{dataset_key}_L"] = [list(item) for item in group_L]
    
    return group_lists

# Example usage
datasets = {
    "I395": "C:\\Users\\Pedram\\Desktop\\PEDRAM - DO NOT TOUCH\\PT_IDM\\I395-final-run-index.csv",
    "I9094": "C:\\Users\\Pedram\\Desktop\\PEDRAM - DO NOT TOUCH\\PT_IDM\\I90_94_moving_final.csv",
    "I294l1": "C:\\Users\\Pedram\\Desktop\\PEDRAM - DO NOT TOUCH\\PT_IDM\\I294_L1_final.csv",
    "I294l2": "C:\\Users\\Pedram\\Desktop\\PEDRAM - DO NOT TOUCH\\PT_IDM\\I294_L2_final.csv"}

vehicle_groups = generate_vehicle_groups(datasets)

# Access generated lists
I395_A = vehicle_groups["I395_A"]
I395_S = vehicle_groups["I395_S"]
I395_L = vehicle_groups["I395_L"]

I9094_A = vehicle_groups["I9094_A"]
I9094_S = vehicle_groups["I9094_S"]
I9094_L = vehicle_groups["I9094_L"]

I294l1_A = vehicle_groups["I294l1_A"]
I294l1_S = vehicle_groups["I294l1_S"]
I294l1_L = vehicle_groups["I294l1_L"]

I294l2_A = vehicle_groups["I294l2_A"]
I294l2_S = vehicle_groups["I294l2_S"]
I294l2_L = vehicle_groups["I294l2_L"]



In [21]:
#testing
print (I9094_A)

[[5366, 1], [286, 3], [195, 2]]


In [22]:
def find_leader_data(df, follower_id, run_index):
    global most_leading_leader_id
    
    follower_data = df[(df['ID'] == follower_id) & (df['run-index'] == run_index)]
    leader_data_dict = {}  # Dictionary to store leader data by leader ID
    
    for index, row in follower_data.iterrows():
        time = row['time']
        follower_x = row[pos]  # Assuming 'pos' is the column for follower's position
        follower_lane = row['lane-kf']  # Assuming the lane information is in 'lane-kf' column
        run_index = row['run-index']  # Assuming 'run-index' is the column for run index

        # Find the nearest leader with a greater position
        leader_data = df[(df['ID'] != follower_id) & (df['time'] == time) & (df['lane-kf'] == follower_lane) & (df[pos] > follower_x) & (df['run-index'] == run_index)]
        
        if not leader_data.empty:
            # Find the nearest leader
            nearest_leader_row = leader_data.loc[leader_data[pos].sub(follower_x).abs().idxmin()]
            
            leader_id = nearest_leader_row['ID']
            leader_x_val = nearest_leader_row[pos]
            leader_speed_val = nearest_leader_row['speed-kf']

            if leader_id not in leader_data_dict:
                leader_data_dict[leader_id] = {'time': [], 'x_val': [], 'speed_val': []}

            leader_data_dict[leader_id]['time'].append(time)
            leader_data_dict[leader_id]['x_val'].append(leader_x_val)
            leader_data_dict[leader_id]['speed_val'].append(leader_speed_val)

    # Choose the most leading leader by selecting the leader with the maximum number of occurrences in the leader_data_dict
    if leader_data_dict:
        most_leading_leader_id = max(leader_data_dict, key=lambda x: len(leader_data_dict[x]['time']))
        leader_data = leader_data_dict[most_leading_leader_id]
        leader_df = pd.DataFrame({'ID': most_leading_leader_id,
                                   'time': leader_data['time'],
                                   pos: leader_data['x_val'],
                                   'speed-kf': leader_data['speed_val'],
                                   'run-index': run_index})
    else:
        leader_df = pd.DataFrame(columns=['ID', 'time', pos, 'speed-kf', 'run-index'])
    
    return leader_df


def extract_subject_and_leader_data(df, follower_id, run_index):
    sdf = df[(df['ID'] == follower_id) & (df['run-index'] == run_index)].round(2)
    ldf = find_leader_data(df, follower_id, run_index).round(2)
    
    # Find the intersection of time frames between leader and subject
    mutual_times = np.intersect1d(ldf['time'], sdf['time'])
    
    # Find the longest continuous segment of mutual time
    max_continuous_mutual_times = []
    continuous_mutual_times = []
    prev_time = None
    for time in mutual_times:
        if prev_time is None or time - prev_time < 0.2:  # Assuming the time step is 0.1
            continuous_mutual_times.append(time)
        else:
            if len(continuous_mutual_times) > len(max_continuous_mutual_times):
                max_continuous_mutual_times = continuous_mutual_times
            continuous_mutual_times = [time]
        prev_time = time
    
    if len(continuous_mutual_times) > len(max_continuous_mutual_times):
        max_continuous_mutual_times = continuous_mutual_times
    
    # Calculate the duration of car-following
    if max_continuous_mutual_times:
        duration = max_continuous_mutual_times[-1] - max_continuous_mutual_times[0]
    else:
        duration = 0  # No mutual time found

    # Filter leader and subject data to include only the longest continuous mutual time
    ldf = ldf[ldf['time'].isin(max_continuous_mutual_times)]
    sdf = sdf[sdf['time'].isin(max_continuous_mutual_times)]
    
    if (isinstance(sdf, list) and not sdf) or (isinstance(sdf, pd.DataFrame) and sdf.empty):
        print(f"No subject data found for Follower ID {follower_id} and Run Index {run_index}.")
        return pd.DataFrame(), pd.DataFrame(), 0  # Return empty DataFrames and duration 0
    
    else:
        start_time = sdf['time'].iloc[0]
        ldf['time'], sdf['time'] = ldf['time'] - start_time, sdf['time'] - start_time
        return sdf, ldf, duration



def idm_acceleration(v, v_leader, s):
    max_v = 40  # Maximum allowable velocity (adjust as needed)
    max_s = 1000  # Maximum allowable gap (adjust as needed)
    
    s_star = so + max (0,(v * T + (v * (v - v_leader)) / (2 * np.sqrt(a * b))))
    #print (s_star)
    acceleration = a * (1 - (v / min(v0, max_v)) ** delta  - (s_star / min(s, max_s))**2)
    if np.isnan(acceleration):
        acceleration = 0
    #print (acceleration)
    return acceleration

def simulate_car_following(params):
    global T, a, b, v0, so, delta
    T, a, b, v0, so, delta = params  # Include delta from the parameters
    
    num_steps = round(total_time / time_step)
    time = np.linspace(0, total_time, num_steps)
    
    position = np.zeros(num_steps)
    speed = np.zeros(num_steps)
    acl = np.zeros(num_steps)
    
    position[0] = sdf.iloc[0][pos]
    speed[0] = sdf.iloc[0]['speed-kf']
    acl[0] = 0

    for i in range(1, num_steps):
        dt = time_step
        desired_position = position[i - 1] + speed[i - 1] * dt
        
        leader_v = leader_speed[i - 1]
        
        # Calculate acceleration using IDM formula
        acceleration = idm_acceleration(speed[i - 1], leader_v, leader_position[i - 1] - position[i - 1])
        
        acl[i] = acceleration
        speed[i] = speed[i - 1] + acceleration * dt
        position[i] = position[i - 1] + speed[i - 1] * dt + 0.5 * acceleration * (dt ** 2)
        
    return position, speed, acl

def fitness(params):
    # Define weights for position and speed errors
    weight_position = 1.0  # Full weight for position
    weight_speed = 0.5     # Half weight for speed

    # Simulate car following
    sim_position, sim_speed, acl = simulate_car_following(params)
    diff_position = np.array(sim_position) - np.array(target_position)
    diff_speed = np.array(sim_speed) - np.array(target_speed)
    
    # Calculate errors with weights
    mse_position = np.mean(diff_position ** 2) * weight_position
    mse_speed = np.mean(diff_speed ** 2) * weight_speed
    mse = mse_position + mse_speed
    
    rmse_position = np.sqrt(mse_position)
    rmse_speed = np.sqrt(mse_speed)
    rmse = np.sqrt(mse)
    
    mae_position = np.mean(np.abs(diff_position)) * weight_position
    mae_speed = np.mean(np.abs(diff_speed)) * weight_speed
    mae = mae_position + mae_speed
    
    mape_position = np.mean(np.abs(diff_position / np.array(target_position))) * 100 * weight_position
    mape_speed = np.mean(np.abs(diff_speed / np.array(target_speed))) * 100 * weight_speed
    mape = (mape_position + mape_speed) / 2
    
    nrmse_position = rmse_position / (np.max(target_position) - np.min(target_position))
    nrmse_speed = rmse_speed / (np.max(target_speed) - np.min(target_speed))
    nrmse = (nrmse_position * weight_position + nrmse_speed * weight_speed) / (weight_position + weight_speed)
    
    sse_position = np.sum(diff_position ** 2) * weight_position
    sse_speed = np.sum(diff_speed ** 2) * weight_speed
    sse = sse_position + sse_speed
    
    ss_res_position = np.sum(diff_position ** 2) * weight_position
    ss_tot_position = np.sum((np.array(target_position) - np.mean(target_position)) ** 2)
    r2_position = 1 - (ss_res_position / ss_tot_position)

    ss_res_speed = np.sum(diff_speed ** 2) * weight_speed
    ss_tot_speed = np.sum((np.array(target_speed) - np.mean(target_speed)) ** 2)
    r2_speed = 1 - (ss_res_speed / ss_tot_speed)
    
    r2 = (r2_position * weight_position + r2_speed * weight_speed) / (weight_position + weight_speed)
    
    total_diff = np.sum(np.abs(diff_position)) * weight_position + np.sum(np.abs(diff_speed)) * weight_speed
    
    # Fitness is the inverse of total error to maximize fitness
    fitness_value = 1.0 / (total_diff + 1e-5)
    
    # Store all error metrics in a dictionary
    error_metrics = {
        'MSE': mse,
        'RMSE': rmse,
        'MAE': mae,
        'MAPE': mape,
        'NRMSE': nrmse,
        'SSE': sse,
        'R-squared': r2,
        'Total Difference': total_diff
    }
    
    return fitness_value, error_metrics  # Return fitness and all error metrics

  
    


def crossover(parent1, parent2):
    crossover_point = random.randint(0, len(parent1) - 1)
    child1 = parent1[:crossover_point] + parent2[crossover_point:]
    child2 = parent2[:crossover_point] + parent1[crossover_point:]
    return child1, child2

def mutate(child):
    for i in range(len(child)):
        if random.random() < mutation_rate:
            child[i] += random.uniform(-0.1, 0.1)
    return child

# Modify genetic_algorithm function
def genetic_algorithm():
    # Define parameter ranges including delta
    t_range = (0.5, 2.5)   # Time headway in seconds
    a_range = (0.3, 3)     # Minimum gap in meters
    b_range = (0.5, 3)     # Comfortable deceleration in m/s^2
    v0_range = (5, 35)     # Desired velocity in m/s
    so_range = (1, 5)      # Acceleration relaxation time in seconds
    delta_range = (3.8, 4.2)  # Exponent in IDM formula

    # Initialize population with random parameter values
    population = [[random.uniform(*range_) for range_ in (t_range, a_range, b_range, v0_range, so_range, delta_range)]
                  for _ in range(population_size)]
    
    best_error = float('inf')  # Initialize best error
    best_individual = None  # Initialize best individual
    best_metrics = None  # Initialize best error metrics
    
    for generation in range(num_generations):
        # Evaluate fitness and errors
        fitness_and_errors = [fitness(individual) for individual in population]
        
        # Sorting population based on fitness (first element of tuple), in descending order
        population_sorted = sorted(zip(population, fitness_and_errors), key=lambda x: x[1][0], reverse=True)
        
        # Update population with sorted individuals
        population = [ind for ind, _ in population_sorted]
        
        # Update best individual and best error if a better one is found
        current_best_error = population_sorted[0][1][1]['Total Difference']  # Error is the second element of the fitness_and_errors tuple
        if current_best_error < best_error:
            best_error = current_best_error
            best_individual = population_sorted[0][0]
            best_metrics = population_sorted[0][1][1]  # Best error metrics
        
        # Parent selection (top half of the sorted population)
        parents = population[:len(population) // 2]
        
        # Generate children via crossover and mutation
        children = []
        while len(children) < (population_size - len(parents)):
            parent1, parent2 = random.sample(parents, 2)
            child1, child2 = crossover(parent1, parent2)
            children.extend([mutate(child1), mutate(child2)])
        
        # The new population consists of the parents and the children
        population = parents + children[:population_size - len(parents)]
    
    # Return the best individual, best error, and best error metrics after all generations
    return best_individual, best_error, best_metrics


def plot_simulation(timex, leader_position, target_position, sim_position, leader_speed, target_speed, sim_speed, follower_id, most_leading_leader_id, run_index, save_dir):
    plt.figure(figsize=(10, 12))
    plt.subplot(2, 1, 1)
    plt.plot(timex, leader_position, label='Leader')
    plt.plot(timex, target_position, label='Target')
    plt.plot(timex, sim_position, label='Simulated Follower')
    plt.xlabel('time (sec)')
    plt.ylabel('Position (m)')
    plt.title(f'Position vs time, FID: {follower_id}, LID: {int(most_leading_leader_id)}, run: {run_index}')
    plt.legend()
    plt.grid(True)
    plt.subplot(2, 1, 2)
    plt.plot(timex, leader_speed, label='Leader')
    plt.plot(timex, target_speed, label='Target')
    plt.plot(timex, sim_speed, label='Simulated Follower')
    plt.xlabel('time (sec)')
    plt.ylabel('Speed (m/s)')
    plt.title(f'Speed vs time, FID: {follower_id}, LID: {int(most_leading_leader_id)}, run: {run_index}')
    plt.legend()
    plt.grid(True)
    plot_filename = os.path.join(save_dir, f'{outname}_FID_{follower_id}_LID_{int(most_leading_leader_id)}_run_{run_index}.png')
    plt.savefig(plot_filename)
    #plt.show()
    plt.close()

def visualize_parameter_distributions(all_params):
    # Assuming all_params is a list of lists, with each inner list containing a set of parameters
    param_names = ['T', 'a', 'b', 'v0', 'so', 'delta']

    num_params = len(param_names)
    
    # Convert list of lists into a 2D numpy array for easier column-wise access
    all_params_array = np.array(all_params)

    print(f"Shape of all_params_array: {all_params_array.shape}")
    print(f"Number of param_names: {len(param_names)}")
    
    # Create histograms for each parameter
    fig, axs = plt.subplots(1, num_params, figsize=(20, 4))
    for i in range(num_params):
        axs[i].hist(all_params_array[:, i], bins=20, color='skyblue', edgecolor='black')
        axs[i].set_title(param_names[i])
        axs[i].set_xlabel('Value')
        axs[i].set_ylabel('Frequency')
    
    plt.tight_layout()
    plt.show()

    # Optionally, create box plots for each parameter
    plt.figure(figsize=(10, 6))
    plt.boxplot(all_params_array, labels=param_names, patch_artist=True)
    plt.title('Distribution of IDM Parameters')
    plt.ylabel('Value')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

In [23]:
import os
import pandas as pd
import numpy as np

datasets = {
    "df395": "C:\\Users\\Pedram\\Desktop\\PEDRAM - DO NOT TOUCH\\PT_IDM\\I395-final-run-index.csv",
    "df9094": "C:\\Users\\Pedram\\Desktop\\PEDRAM - DO NOT TOUCH\\PT_IDM\\I90_94_moving_final.csv",
    "df294l1": "C:\\Users\\Pedram\\Desktop\\PEDRAM - DO NOT TOUCH\\PT_IDM\\I294_L1_final.csv",
    "df294l2": "C:\\Users\\Pedram\\Desktop\\PEDRAM - DO NOT TOUCH\\PT_IDM\\I294_L2_final.csv"}

groups = {
    "df395": ["I395_A", "I395_S", "I395_L"],
    "df9094": ["I9094_L", "I9094_S", "I9094_A"],
    "df294l1": ["I294l1_L", "I294l1_S", "I294l1_A"],
    "df294l2": ["I294l2_L", "I294l2_S", "I294l2_A"]
}

# Save directory for plots
save_dir = "C:\\Users\\Pedram\\Desktop\\PEDRAM - DO NOT TOUCH\\PT_IDM\\Plots\\"

# Ensure save directory exists
os.makedirs(save_dir, exist_ok=True)

# Iterate through each dataset and group
for df_key, df_path in datasets.items():
    df = pd.read_csv(df_path)
    df = df.sort_values(by='time')
    df['time'] = df['time'].round(1)
    pos = "yloc-kf" if df_key == "df395" else "xloc-kf"
    
    for group in groups[df_key]:
        # Define the output CSV file name for this group
        outname = f"IDM_{group}.csv"
        output_csv_path = os.path.join(save_dir, outname)
        
        # Check if the CSV file already exists
        if os.path.exists(output_csv_path):
            print(f"File {output_csv_path} already exists. Skipping...")
            continue  # Skip this group
        
        # Define the current group
        AVs = eval(group)  # Assuming `group` is a string that evaluates to a list of tuples
        all_params = []
        params_list = []
        
        
        for data in AVs:
            follower_id, run_index = data
            sdf, ldf, duration = extract_subject_and_leader_data(df, follower_id, run_index)
            print(f"Processing Follower ID: {follower_id}, Run Index: {run_index}")
        
            # Check if sdf is empty
            if sdf.empty:
                print(f"No data found for Follower ID {follower_id} and Run Index {run_index}. Skipping...")
                continue

            total_time = len(ldf) * 0.1
            time_step, num_steps = 0.1, round(total_time / 0.1)
            timex = np.linspace(0, total_time, num_steps)
            leader_position, leader_speed = ldf[pos].tolist(), ldf['speed-kf'].tolist()
            target_position, target_speed = sdf[pos].tolist(), sdf['speed-kf'].tolist()
            
            best_params, best_error, best_metrics = genetic_algorithm()
        
            if best_params is None or best_error is None or not best_metrics:
                print(f"Skipping Follower ID {follower_id}, Run Index {run_index} due to missing optimization results.")
                continue
        
            all_params.append(best_params)
            
            # Add `duration` after `Run_Index`
            params_list.append([follower_id, run_index, duration] + best_params + [best_error] + list(best_metrics.values()))
            
            sim_position, sim_speed, acl = simulate_car_following(best_params)
            plot_simulation(
                timex, leader_position, target_position, sim_position,
                leader_speed, target_speed, sim_speed, follower_id,
                most_leading_leader_id, run_index, save_dir
            )

        # Modify CSV column names to include `Duration`
        metrics_names = list(best_metrics.keys())
        columns = ['Follower_ID', 'Run_Index', 'Duration', 'T', 'a', 'b', 'v0', 'so', 'delta', 'Error'] + metrics_names
        params_df = pd.DataFrame(params_list, columns=columns)
        params_df.to_csv(output_csv_path, index=False)
        print(f"Saved results for {group} to {output_csv_path}")

Processing Follower ID: 694.0, Run Index: 1.0
Processing Follower ID: 5741.0, Run Index: 1.0
Processing Follower ID: 3758.0, Run Index: 1.0
Processing Follower ID: 10182.0, Run Index: 1.0
Processing Follower ID: 1416.0, Run Index: 1.0
Processing Follower ID: 7299.0, Run Index: 1.0
Processing Follower ID: 8692.0, Run Index: 1.0
Processing Follower ID: 1779.0, Run Index: 1.0
Processing Follower ID: 4715.0, Run Index: 1.0
Processing Follower ID: 4261.0, Run Index: 1.0
Processing Follower ID: 3330.0, Run Index: 1.0
Processing Follower ID: 2342.0, Run Index: 1.0
Processing Follower ID: 8885.0, Run Index: 1.0
Processing Follower ID: 9409.0, Run Index: 1.0


C:\Users\Pedram\AppData\Local\Temp\ipykernel_7184\3823207850.py:96: RuntimeWarning: invalid value encountered in scalar power
  acceleration = a * (1 - (v / min(v0, max_v)) ** delta  - (s_star / min(s, max_s))**2)


Processing Follower ID: 2817.0, Run Index: 1.0
Processing Follower ID: 6398.0, Run Index: 1.0
Processing Follower ID: 10150.0, Run Index: 1.0
Processing Follower ID: 5385.0, Run Index: 1.0
Processing Follower ID: 9505.0, Run Index: 1.0
Processing Follower ID: 8171.0, Run Index: 1.0
Processing Follower ID: 7832.0, Run Index: 1.0
Saved results for I395_A to C:\Users\Pedram\Desktop\PEDRAM - DO NOT TOUCH\PT_IDM\Plots\IDM_I395_A.csv
Processing Follower ID: 5338.0, Run Index: 1.0
Processing Follower ID: 5472.0, Run Index: 1.0
Processing Follower ID: 5076.0, Run Index: 1.0
Processing Follower ID: 10585.0, Run Index: 1.0
Processing Follower ID: 4814.0, Run Index: 1.0
Processing Follower ID: 7550.0, Run Index: 1.0
Processing Follower ID: 5551.0, Run Index: 1.0
Processing Follower ID: 5289.0, Run Index: 1.0
Processing Follower ID: 4893.0, Run Index: 1.0
Processing Follower ID: 5027.0, Run Index: 1.0
Processing Follower ID: 10536.0, Run Index: 1.0


C:\Users\Pedram\AppData\Local\Temp\ipykernel_7184\3823207850.py:96: RuntimeWarning: invalid value encountered in scalar power
  acceleration = a * (1 - (v / min(v0, max_v)) ** delta  - (s_star / min(s, max_s))**2)


Processing Follower ID: 5764.0, Run Index: 1.0
Processing Follower ID: 8537.0, Run Index: 1.0
Processing Follower ID: 5502.0, Run Index: 1.0
Processing Follower ID: 5106.0, Run Index: 1.0
Processing Follower ID: 10615.0, Run Index: 1.0
Processing Follower ID: 5240.0, Run Index: 1.0
Processing Follower ID: 4844.0, Run Index: 1.0
Processing Follower ID: 10353.0, Run Index: 1.0
Processing Follower ID: 2845.0, Run Index: 1.0
Processing Follower ID: 8750.0, Run Index: 1.0
Processing Follower ID: 5319.0, Run Index: 1.0


...

Processing Follower ID: 291, Run Index: 28
Processing Follower ID: 218, Run Index: 35


C:\Users\Pedram\AppData\Local\Temp\ipykernel_7184\3823207850.py:96: RuntimeWarning: invalid value encountered in scalar power
  acceleration = a * (1 - (v / min(v0, max_v)) ** delta  - (s_star / min(s, max_s))**2)


Processing Follower ID: 211, Run Index: 41


C:\Users\Pedram\AppData\Local\Temp\ipykernel_7184\3823207850.py:96: RuntimeWarning: invalid value encountered in scalar power
  acceleration = a * (1 - (v / min(v0, max_v)) ** delta  - (s_star / min(s, max_s))**2)


Processing Follower ID: 462, Run Index: 5
Processing Follower ID: 72, Run Index: 38
Saved results for I294l2_A to C:\Users\Pedram\Desktop\PEDRAM - DO NOT TOUCH\PT_IDM\Plots\IDM_I294l2_A.csv


In [ ]:
#### FOR SIMULATION #### DO NOT RUN

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Load leader data
leader_data_path, ID, run_index = "/Users/pedrambeigi/Desktop/BAA/Vehicles Modeling/Codes/I294_L1_final.csv", 364, 8
leader_data = pd.read_csv(leader_data_path)
leader_data = leader_data[(leader_data['ID'] == ID) & (leader_data['run-index'] == run_index)]
leader_velocity = leader_data['speed-kf'].values
leader_time = leader_data['time'].values

# Provided data
velocities = leader_velocity.copy()
times = leader_time.copy()

# Define stop-and-go times
time_start = 60
time_stop = 64
time_zero = 74
time_resume = 82

# Find indices corresponding to specified times
indices_start = np.where(times == time_start)[0][0]
indices_stop = np.where(times == time_stop)[0][0]
indices_zero = np.where(times == time_zero)[0][0]
indices_resume = np.where(times == time_resume)[0][0]

# Modify velocities to simulate stop-and-go
# Gradually decrease velocity to zero
velocities[indices_start:indices_stop+1] = np.linspace(velocities[indices_start], 0, indices_stop-indices_start+1)

# Keep velocity zero for the specified time period
velocities[indices_stop:indices_zero+1] = 0

# Gradually increase velocity back to original value
velocities[indices_zero:indices_resume+1] = np.linspace(0, velocities[indices_resume], indices_resume-indices_zero+1)

# Recompute positions based on the new velocities, starting from an initial position of 0
positions_updated = np.zeros_like(velocities)
for i in range(1, len(times)):
    positions_updated[i] = positions_updated[i-1] + velocities[i-1] * (times[i] - times[i-1])

# Prepare the data for the DataFrame
data = {
    'time': times,
    'yloc-kf': positions_updated,
    'speed-kf': velocities,
    'ID': 1,
    'run-index': 1
}

# Create DataFrame and save to CSV
df = pd.DataFrame(data)
df.to_csv('updated_traffic_data.csv', index=False)

# Plot the position and velocity vs. time
plt.figure(figsize=(12, 6))

# Position plot
plt.subplot(2, 1, 1)
plt.plot(times, positions_updated, label='Position (xloc-kf)', color='blue')
plt.xlabel('Time (s)')
plt.ylabel('Position (m)')
plt.title('Position vs. Time with Artificial Stop-and-Go')
plt.legend()
plt.grid(True)
# Velocity plot
plt.subplot(2, 1, 2)
plt.plot(times, velocities, label='Velocity (speed-kf)', color='red')
plt.xlabel('Time (s)')
plt.ylabel('Velocity (m/s)')
plt.title('Velocity vs. Time with Artificial Stop-and-Go')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig('/Users/pedrambeigi/Desktop/BAA/Vehicles Modeling/Codes/Results/Stop_Go.png')
plt.show()

In [ ]:
import numpy as np

def generate_scaled_lognormal(mean, std, size, scale, min_val, max_val):
    mu = np.log((mean ** 2) / np.sqrt(std ** 2 + mean ** 2))
    sigma = np.sqrt(np.log((std ** 2) / (mean ** 2) + 1))
    lognormal_vals = np.random.lognormal(mu, sigma, size) * scale
    return np.clip(lognormal_vals, min_val, max_val).tolist()
# Parameters
params = [
    (2.63, 2.8, 300),
    (1.43, 1.7, 300),
    (1.50, 1.8, 300)]
scale = 20.52
min_val = 10
max_val = 50

#Generate and scale lognormal distributions
lognormal_dist1 = generate_scaled_lognormal(*params[0], scale, min_val, max_val)
lognormal_dist2 = generate_scaled_lognormal(*params[1], scale, min_val, max_val)
lognormal_dist3 = generate_scaled_lognormal(*params[2], scale, min_val, max_val)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as mcolors

# IDM parameters and distances
params_list = [
    {"T": 1.44, "a": 1.85, "b": 1.56, "v0": 28.52, "s0": 3.31, "delta": 4, "distances": lognormal_dist2, "title": "Positions of Cars Over Time, Small Vehicles"},
    {"T": 1.69, "a": 1.82, "b": 1.65, "v0": 22.79, "s0": 3.31, "delta": 4, "distances": lognormal_dist1, "title": "Positions of Cars Over Time, Large Vehicles"},
    {"T": 1.49, "a": 1.50, "b": 1.62, "v0": 27.09, "s0": 3.49, "delta": 4, "distances": lognormal_dist2, "title": "Positions of Cars Over Time, Autonomous Vehicles"}]

# Leader data parameters
#leader_data_path, ID, run_index = "/Users/pedrambeigi/Desktop/BAA/Vehicles Modeling/Codes/I294_L1_final.csv", 364, 8
#leader_data_path, ID, run_index = "/Users/pedrambeigi/Desktop/BAA/Vehicles Modeling/Codes/I395-final-run-index.csv", 694, 1
leader_data_path, ID, run_index = 'updated_traffic_data.csv', 1, 1
t1, t2, num_cars, time_step, limit_time = 30, 100, 200, 0.1, False

# Read and filter leader data
leader_data = pd.read_csv(leader_data_path)
leader_data = leader_data[(leader_data['ID'] == ID) & (leader_data['run-index'] == run_index)]
leader_position = leader_data['yloc-kf'].values
leader_velocity = leader_data['speed-kf'].values
leader_time = leader_data['time'].values

# Simulation time setup
if limit_time:
    leader_mask = (leader_time >= t1) & (leader_time <= t2)
    leader_position = leader_position[leader_mask]
    leader_velocity = leader_velocity[leader_mask]
    leader_time = leader_time[leader_mask]
    time_points = np.arange(t1, t2, time_step)
else:
    time_points = np.arange(leader_time[0], leader_time[-1], time_step)

def idm_acceleration(v, v_leader, s, T, a, b, v0, s0, delta):
    max_v, max_s = 50, 50
    s_star = s0 + max(0, (v * T + (v * (v - v_leader)) / (2 * np.sqrt(a * b))))
    return a * (1 - (v / min(v0, max_v))**delta - (s_star / min(s, max_s))**2)

# Initialize a DataFrame to store the results
columns = ['id', 'time', 'loc', 'speed', 'params_list_index']
trajectories_df = pd.DataFrame(columns=columns)

# Running simulations for each parameter set
for i, params in enumerate(params_list):
    T, a, b, v0, s0, delta, distances, title = params.values()
    # Initialize positions and velocities
    positions = np.zeros((num_cars, len(time_points)))
    velocities = np.zeros((num_cars, len(time_points)))
    for j in range(num_cars):
        positions[j, 0] = leader_position[0] - sum(distances[:j+1])
        velocities[j, 0] = leader_velocity[0]
    
    # Simulation loop
    for t in range(1, len(time_points)):
        for j in range(num_cars):
            if j == 0:
                leader_idx = min(np.searchsorted(leader_time, time_points[t]), len(leader_position) - 1)
                v_leader = leader_velocity[leader_idx]
                s = leader_position[leader_idx] - positions[j, t-1]
            else:
                v_leader = velocities[j-1, t-1]
                s = positions[j-1, t-1] - positions[j, t-1]
            
            velocities[j, t] = velocities[j, t-1] + idm_acceleration(velocities[j, t-1], v_leader, s, T, a, b, v0, s0, delta) * time_step
            positions[j, t] = positions[j, t-1] + velocities[j, t] * time_step
            
            if positions[j, t] < positions[j, t-1]:
                positions[j, t] = positions[j, t-1]

    # Save the results to the DataFrame
    for j in range(num_cars):
        car_id = j
        car_trajectory = pd.DataFrame({
            'ID': car_id,
            'time': time_points,
            'yloc-kf': positions[j, :],
            'speed-kf': velocities[j, :],
            'params_list_index': i,
            'lane-kf': 1
        })
        trajectories_df = pd.concat([trajectories_df, car_trajectory])
print(trajectories_df.head())
trajectories_df.to_csv('trajectories.csv', index=False)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as mcolors

# Load the trajectories data
trajectories_df = pd.read_csv('trajectories.csv')

# List of titles for each params_list_index
titles = [
    "Positions of Cars Over Time, Small Vehicles",
    "Positions of Cars Over Time, Large Vehicles",
    "Positions of Cars Over Time, Autonomous Vehicles"]

# Plotting function
def plot_trajectories(data, title, file_name):
    # Filter data for the given parameter set index
    leader_data = data[data['ID'] == 0]
    time_points = leader_data['time'].values
    leader_position = leader_data['yloc-kf'].values
    
    # Normalize the colormap to the range of speeds
    norm = mcolors.Normalize(vmin=data['speed-kf'].min(), vmax=data['speed-kf'].max())
    #cmap = cm.viridis
    #cmap = cm.RdBu

    from matplotlib.colors import LinearSegmentedColormap
    # Define a custom colormap with sharp red-to-green transitions
    norm = mcolors.Normalize(vmin=0, vmax=30)
    colors = [
        (1, 0, 0),  # Red (low red)
        (1, 0, 0),  # High red
        (0, 1, 0),  # Low green
        (0, 1, 0)]  # High green]
    cmap = LinearSegmentedColormap.from_list("RedGreenTransition", colors, N=256)

    
    # Create a figure and a set of subplots with defined layout
    fig, ax = plt.subplots(figsize=(18, 6))
    
    # Plot leader's trajectory
    ax.plot(time_points, leader_position, color='black', linestyle='dashed')#, label='Leader Car')
    
    # Plot each car's trajectory
    scatter = None
    for car_id in data['ID'].unique():
        if car_id == 0:
            continue
        car_data = data[data['ID'] == car_id]
        scatter = ax.scatter(car_data['time'], car_data['yloc-kf'], c=car_data['speed-kf'], cmap=cmap, norm=norm, s=1)
    
    # Set labels and title
    ax.set_xlabel('Time (s)')
    ax.set_ylabel('Position (m)')
    ax.set_title(title)
    cbar = plt.colorbar(scatter, ax=ax, pad=0.001, label='Speed (m/s)')  # pad controls the space between the plot and colorbar
    ax.grid(True)
    #ax.legend(loc='upper left', bbox_to_anchor=(1, 1))
    #plt.xlim(0, 300)
    #plt.ylim(0, 1000)
    #ax.invert_yaxis()

    fig.tight_layout()  # Adjust layout to make room for legends and colorbar
    plt.savefig(file_name)
    plt.show()

# Generate plots for each parameter set
for index in range(len(titles)):
    params_data = trajectories_df[trajectories_df['params_list_index'] == index]
    file_name = f'/Users/pedrambeigi/Desktop/BAA/Vehicles Modeling/Codes/Results/trajectories_plot_{index}.png'  # Define the file name for each plot
    plot_trajectories(params_data, titles[index], file_name)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Load the trajectories data
trajectories_df = pd.read_csv('trajectories.csv')
#trajectories_df = trajectories_df[(trajectories_df['time'] >= 100) & (trajectories_df['time'] <= 200)]

# Define the detector positions
detector_positions = np.arange(-4000, 2501, 50)
# Define the time interval for flow calculation
time_interval = 60  # seconds
# Initialize a list to store the results
results_list = []

# Analyze each params_list_index separately
for params_index in [0, 1, 2]:
    # Filter the data for the current params_list_index
    params_df = trajectories_df[trajectories_df['params_list_index'] == params_index]
    
    # Determine the time range
    min_time = params_df['time'].min()
    max_time = params_df['time'].max()
    
    # Calculate the flow, speed, and density for each detector
    for detector in detector_positions:
        for start_time in np.arange(min_time, max_time, time_interval):
            end_time = start_time + time_interval
            
            # Filter cars that pass the detector within the time interval
            passed_cars = params_df[
                (params_df['time'] >= start_time) &
                (params_df['time'] < end_time) &
                (
                    ((params_df['yloc-kf'] > detector) & (params_df['yloc-kf'].shift(1) <= detector)) |
                    ((params_df['yloc-kf'] < detector) & (params_df['yloc-kf'].shift(1) >= detector))
                )]
            
            # Calculate the flow (number of cars)
            flo = passed_cars['ID'].nunique()
            flow = flo * (3600/time_interval)
            
            # Calculate the average speed of the passing cars
            if flow > 0:
                average_speed = passed_cars['speed-kf'].mean()
            else:
                average_speed = 1
            
            # Calculate density as the number of cars per unit distance at the detector (cars/m)
            # Here, we assume a simple uniform density distribution model for demonstration purposes
            # You can adjust the density calculation based on more specific criteria or data
            density = 10 * flo / (average_speed) # Assume density is cars per meter (per unit distance)

            # Store the results
            result = {
                'params_list_index': params_index,
                'detector_position': detector,
                'start_time': start_time,
                'end_time': end_time,
                'flow': flow,
                'average_speed': average_speed,
                'density': density
            }
            
            results_list.append(result)

# Convert the results list to a DataFrame
results_df = pd.DataFrame(results_list)
results_df = results_df[results_df['average_speed'] <= 20]
results_dfs = results_df[~((results_df['density'] < 0.1) & (results_df['average_speed'] < 15))]


# Remove points manually based on density and flow ranges
# Small Vehicles
results_df = results_df[~(
    ((results_df['params_list_index'] == 0) & (results_df['density'].between(15, 18)) & (results_df['flow'].between(2000, 2200))) |
    ((results_df['params_list_index'] == 0) & (results_df['density'].between(20, 30)) & (results_df['flow'].between(1300, 1500))))]
# Large Vehicles
results_df = results_df[~(
    ((results_df['params_list_index'] == 1) & (results_df['density'].between(15, 30)) & (results_df['flow'].between(1000, 1300))))]

titles = [
    "Flow-density relation for Small Vehicles",
    "Flow-density relation for Large Vehicles",
    "Flow-density relation for Autonomous Vehicles"
]

# Create a 1x3 subplot layout
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Iterate over each params_list_index
for idx, params_index in enumerate([0, 1, 2]):
    # Filter data for each params_list_index
    plot_df = results_df[results_df['params_list_index'] == params_index]
    
    # Plot Density vs Flow for each params_list_index
    axes[idx].scatter(plot_df['density'], plot_df['flow'], c='blue')#, label=f'Params {params_index}')
    axes[idx].set_title(titles[idx])  # Set the title from the list
    axes[idx].set_xlabel('Density (Veh/km)')
    axes[idx].set_ylabel('Flow (Veh/hr)')
    axes[idx].set_xlim(0, 80)
    axes[idx].set_ylim(0, 2500)
    axes[idx].legend()

# Adjust layout to prevent overlap
plt.tight_layout()
plt.savefig(file_name)
plt.show()

"""
# Plotting
for params_index in [0, 1, 2]:
    # Filter data for each params_list_index
    plot_df = results_df[results_df['params_list_index'] == params_index]
    
    # Create a 3x1 subplot
    fig, axes = plt.subplots(1, 3, figsize=(12, 3))
    
    # Plot Density vs Flow
    axes[0].scatter(plot_df['density'], plot_df['flow'], c='blue')
    axes[0].set_title(f'Density vs Flow ({params_index})')
    axes[0].set_xlabel('Density (cars/m)')
    axes[0].set_ylabel('Flow (vehicles/hour)')
    axes[0].set_xlim(0, 80)
    axes[0].set_ylim(0, 2500)
    axes[0].legend()
    
    # Plot Density vs Speed
    axes[1].scatter(plot_df['density'], plot_df['average_speed'], c='green')
    axes[1].set_title(f'Density vs Speed ({params_index})')
    axes[1].set_xlabel('Density (cars/m)')
    axes[1].set_ylabel('Average Speed (m/s)')
    axes[1].set_xlim(0, 50)
    axes[1].set_ylim(0, 22)
    axes[1].legend()
    
    # Plot Flow vs Speed
    axes[2].scatter(plot_df['flow'], plot_df['average_speed'], c='red')
    axes[2].set_title(f'Flow vs Speed ({params_index})')
    axes[2].set_xlabel('Flow (vehicles/hour)')
    axes[2].set_ylabel('Average Speed (m/s)')
    axes[2].set_xlim(0, 2500)
    axes[2].set_ylim(0, 22)
    axes[2].legend()
    
    plt.tight_layout()
    plt.show()
"""